In [1]:
import tensorflow as tf
from keras.layers import Input, Conv1D, LSTM, Dense, Reshape, Concatenate, Attention, Reshape, Input, Dropout, MultiHeadAttention
from keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('')

FileNotFoundError: [Errno 2] No such file or directory: '/limit_order_book/truncated_LOB_data_BTC_USD_COINBASE.csv'

In [ ]:
def finder_of_fulfilment(df, column):
    counter = []
    for i in range(len(df)):
        num = df[column].iloc[i]
        arr = df[df[column]>num][column].index
        counter.append(df['timestamp'].iloc[arr[arr>i][0]] if len(arr[arr>i]) else -1)
    return counter

In [ ]:
df['counter'] = finder_of_fulfilment(df, 'ask_prices_0')
df

In [ ]:
df = df[df['counter'] != -1]

In [ ]:
df['counter'] = pd.to_datetime(df['counter'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['exec_time'] = (df['counter'] - df['timestamp']).dt.total_seconds().astype(int)
df = df.iloc[0:-140]

In [ ]:
#Make some features and drop useless columns 
df.drop(columns = ['timestamp', 'counter'], inplace=True)
pd.options.mode.chained_assignment = None

df['vol_imbalance'] = (df['bid_quantity_0'] - df['ask_quantity_0'])/(df['bid_quantity_0'] + df['ask_quantity_0'])
df['microprice'] = ((df['bid_prices_0']*df['bid_quantity_0']+df['ask_prices_0']*df['ask_quantity_0'])
                    /(df['bid_quantity_0'] + df['ask_quantity_0']))

In [ ]:
figure = plt.figure(figsize=(10,6))
plt.plot(df['microprice'])
plt.show()

In [ ]:
X = df.drop(columns=['exec_time'])
Y = df['exec_time']

# Simplified Staff

In [ ]:
input_shape = X.shape[1:]  # Assuming X is your data
steps = 1  # You need to determine the appropriate number of steps based on your data
new_input_shape = (input_shape[0], steps)
encoder_input = Input(shape=input_shape)
encoder_reshape = Reshape(new_input_shape)(encoder_input)
encoder_conv1 = Conv1D(32, kernel_size=2, activation='relu')(encoder_reshape)
encoder_lstm = LSTM(32, activation='relu')(encoder_conv1)
latent_dim = 32
encoder_output = Dense(latent_dim)(encoder_lstm)

encoder_model = Model(encoder_input, encoder_output)

# Define the decoder
decoder_input = Input(shape=(latent_dim,))
decoder_dense1 = Dense(32, activation='relu')(decoder_input)
decoder_reshape = Reshape((1, 32))(decoder_dense1)
decoder_conv1 = Conv1D(32, kernel_size=3, activation='relu', padding='same')(decoder_reshape)
output_dim = 32
decoder_output = Dense(output_dim, activation='sigmoid')(decoder_conv1)

decoder_model = Model(decoder_input, decoder_output)

# Combine the encoder and decoder into an autoencoder
autoencoder_input = Input(shape=input_shape)
encoded = encoder_model(autoencoder_input)
decoded = decoder_model(encoded)
autoencoder_model = Model(autoencoder_input, decoded)

# Compile the model
autoencoder_model.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder_model.fit(X, Y, epochs=3, batch_size=32)

# some experiements 

In [ ]:
# Define the DCC layer (updated version)
class DilatedCausalConvolution(layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rate):
        super().__init__()
        self.query_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                        dilation_rate=dilation_rate, padding='causal', activation='relu')
        self.key_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                      dilation_rate=dilation_rate, padding='causal', activation='relu')
        self.value_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                        dilation_rate=dilation_rate, padding='causal', activation='relu')

    def call(self, inputs):
        query = self.query_conv(inputs)
        key = self.key_conv(inputs)
        value = self.value_conv(inputs)
        return query, key, value

# Define the Transformer block (updated version)
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, d_model):
        super().__init__()
        self.multi_head_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dense_proj = layers.Dense(d_model, activation='relu')

    def call(self, query, key, value):
        attn_output = self.multi_head_attention(query, key, value)
        proj_output = self.dense_proj(attn_output)
        return proj_output

# Define the encoder using the DCC and Transformer block (updated version)
def create_encoder(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model):
    inputs = layers.Input(shape=input_shape)
    dcc_layer = DilatedCausalConvolution(filters=filters, kernel_size=kernel_size, dilation_rate=dilation_rate)
    query, key, value = dcc_layer(inputs)
    transformer_block = TransformerBlock(num_heads=num_heads, d_model=d_model)
    transformer_output = transformer_block(query, key, value)
    model = models.Model(inputs=inputs, outputs=transformer_output)
    return model

# Define the monotonic decoder (simplified version)
def create_monotonic_decoder(input_shape, output_shape):
    inputs = layers.Input(shape=input_shape)
    # The decoder would have constraints to ensure monotonicity, omitted here for simplicity
    outputs = layers.Dense(output_shape, activation='sigmoid')(inputs)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Combine the encoder and decoder to create the full model (updated version)
def create_full_model(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model, output_shape):
    encoder_inputs = layers.Input(shape=input_shape)
    encoder = create_encoder(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model)
    encoder_output = encoder(encoder_inputs)
    
    # Assuming the encoder_output has shape (batch_size, seq_length, d_model)
    # If the next layer expects a flat vector, we can flatten the encoder output
    # Otherwise, adjust the shape according to the requirements of your decoder
    flattened_output = layers.Flatten()(encoder_output)
    
    decoder = create_monotonic_decoder(flattened_output.shape[1:], output_shape)
    decoder_output = decoder(flattened_output)
    
    full_model = models.Model(inputs=encoder_inputs, outputs=decoder_output)
    return full_model

In [ ]:
# Expand dimensions of X to add the channel dimension if needed
X_expanded = np.expand_dims(X, axis=-1)  # Shape becomes (99601, 22, 1)

# Define model parameters
input_shape = (22, 1)  # 22 timesteps and 1 feature per timestep
filters = 64
kernel_size = 3
dilation_rate = 1
num_heads = 8
d_model = 64
output_shape = 1

# Create the full model
model = create_full_model(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model, output_shape)

# Compile the model (define loss and optimizer)
model.compile(optimizer='adam', loss='MSE')

# Fit the model with the expanded input data
model.fit(X_expanded, Y, epochs=10, batch_size=32)

# Print the model summary to verify
model.summary()